# Lesson 6: Connecting the MCP Chatbot to Reference Servers 

이제 MCP chatbot이 모든 MCP server에 연결할 수 있도록 하여 chatbot 기능을 확장할 것입니다. 이전 레슨에서 구축한 research server의 tool들에 더해, 두 개의 공식 MCP server tool들을 통합할 것입니다.

<img src="images/lesson_6.png" width="400">

## Open-Source MCP Servers

이 [repo](https://github.com/modelcontextprotocol/servers)에서 MCP server들의 reference implementation 모음뿐만 아니라 community에서 구축한 server들에 대한 reference와 추가 resource들을 찾을 수 있습니다. 두 개의 reference server를 사용하여 MCP chatbot에 해당 tool들을 통합할 것입니다:
- [fetch](https://github.com/modelcontextprotocol/servers/tree/main/src/fetch): 인터넷에서 URL을 fetch하고 내용을 markdown으로 추출하는 `fetch` tool을 제공합니다.
- [filesystem](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem): 지정한 directory 내의 file들과 directory들과 상호작용하기 위한 여러 tool들을 제공합니다.

각 server가 노출하는 기능들과 실행 방법을 확인하려면 각 server의 readme file을 확인할 수 있습니다.

## Updating the MCP Chatbot - Optional Reading

Here are the updates you'll make to the chatbot. You're encouraged to read this section before or after you watch the video, if you'd like to learn more about the details of the code.

- chatbot에서 server parameter들을 하드코딩하는 대신, chatbot이 JSON file에서 server configuration들을 읽어올 것입니다:
  ### Server Configuration
  In the `L6/mcp_project`, you can find the `server_config.json` configuration file that has the following structure.
    ``` json
    {
        "mcpServers": {
            
            "filesystem": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    "."
                ]
            },
            
            "research": {
                "command": "uv",
                "args": ["run", "research_server.py"]
            },
            
            "fetch": {
                "command": "uvx",
                "args": ["mcp-server-fetch"]
            }
        }
    }
    ```
reference server들의 경우, `npx`와 `uvx` command들이 server file들을 로컬 environment에 직접 설치합니다(미리 설치할 필요가 없습니다). `filesystem`의 경우 `.`이 세 번째 argument로 제공되며 이는 "현재 directory"를 의미합니다. 즉, `fetch` server가 현재 directory 내의 file들과 directory들과 상호작용할 수 있도록 허용하는 것입니다.

In [1]:
%%writefile mcp_project/server_config.json
{
    "mcpServers": {

        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "."
            ]
        },

        "research": {
            "command": "uv",
            "args": ["run", "research_server.py"]
        },

        "fetch": {
            "command": "uvx",
            "args": ["mcp-server-fetch"]
        }
    }
}

Writing mcp_project/server_config.json


- 다음은 업데이트된 MCP_chatbot의 대략적인 diagram입니다:
  
  <img src="images/updated_class.png" width="600">

  1. 하나의 session을 갖는 대신, 이제 각 client session이 각 server와 1대1 연결을 설정하는 client session들의 list를 가집니다.
  2. `available_tools`에는 chatbot이 연결할 수 있는 모든 server들에 의해 노출되는 모든 tool들의 정의가 포함됩니다.
  3. `tool_to_session`은 tool 이름을 해당 client session에 mapping합니다. 이렇게 하면 LLM이 특정 tool 이름을 결정할 때, 올바른 client session에 mapping하여 해당 session을 사용해 올바른 MCP server에 `tool_call` request를 보낼 수 있습니다.
  4. `exit_stack`은 mcp client object들과 그들의 session들을 관리하고 적절히 닫힌다는 것을 보장하는 context manager입니다. 레슨 5에서는 `with` statement를 사용했기 때문에 이를 사용하지 않았습니다. `with` statement는 내부적으로 context manager를 사용합니다. 여기서도 다시 `with` statement를 사용할 수 있지만, 연결할 server가 여러 개이므로 여러 개의 중첩된 `with` statement를 사용하게 될 수 있습니다. `exit_stack`을 사용하면 아래 code에서 보게 될 것처럼 mcp client들과 그들의 session들을 동적으로 추가할 수 있습니다.
  5. `connect_to_servers`는 server configuration file을 읽고 각 개별 server에 대해 helper method인 `connect_to_server`를 호출합니다. 이 후자 method에서는 MCP client가 생성되어 server를 sub-process로 실행하는 데 사용되고, 그 다음 client session이 생성되어 server에 연결하고 server가 제공하는 tool들의 list 설명을 가져옵니다.
  6. `cleanup`은 모든 connection들이 작업을 마쳤을 때 적절히 종료되도록 보장하는 helper method입니다. 레슨 5에서는 resource를 자동으로 정리하기 위해 `with` statement에 의존했습니다. 이 cleanup method는 비슷한 목적을 제공하지만, exit_stack에 추가한 모든 resource들에 대해서입니다. 이는 접시를 쌓고 언스택하는 것처럼 추가된 역순으로 (MCP client들과 session들을) 닫습니다. 이는 resource leak을 피하기 위해 network programming에서 특히 중요합니다.

## Updated Code for the MCP Chatbot

In [4]:
%%writefile mcp_project/mcp_chatbot.py

from dotenv import load_dotenv
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict
from contextlib import AsyncExitStack
import json
import asyncio
import os

load_dotenv()

# Set the API key from environment variable
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')

class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.sessions: List[ClientSession] = []
        self.exit_stack = AsyncExitStack()
        self.client = genai.Client()
        self.chat = None  # Chat session for maintaining context
        self.available_tools: List[ToolDefinition] = []
        self.tool_to_session: Dict[str, ClientSession] = {}

    def simplify_schema_for_genai(self, schema):
        """Convert MCP schema to a simpler format that Google genai accepts"""
        if isinstance(schema, dict):
            simplified = {"type": "object", "properties": {}}

            # Handle the root object
            if "properties" in schema:
                for prop_name, prop_def in schema["properties"].items():
                    if isinstance(prop_def, dict):
                        prop_type = prop_def.get("type", "string")
                        simplified_prop = {"type": prop_type}

                        if "description" in prop_def:
                            simplified_prop["description"] = prop_def["description"]

                        # Handle array types simply
                        if prop_type == "array" and "items" in prop_def:
                            items_type = prop_def["items"].get("type", "string")
                            simplified_prop["items"] = {"type": items_type}

                        simplified["properties"][prop_name] = simplified_prop

            # Add required fields if they exist
            if "required" in schema:
                simplified["required"] = schema["required"]

            return simplified
        else:
            return schema

    def convert_mcp_tools_to_genai_format(self, mcp_tools):
        """Convert MCP tools to Gemini format"""
        gemini_tools = []
        for tool in mcp_tools:
            # Simplify the input schema for Google genai compatibility
            simplified_schema = self.simplify_schema_for_genai(tool["input_schema"])

            gemini_tool = {
                "name": tool["name"],
                "description": tool["description"],
                "parameters": simplified_schema
            }
            gemini_tools.append(gemini_tool)
        return gemini_tools

    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """Connect to a single MCP server."""
        try:
            server_params = StdioServerParameters(**server_config)
            stdio_transport = await self.exit_stack.enter_async_context(
                stdio_client(server_params)
            )
            read, write = stdio_transport
            session = await self.exit_stack.enter_async_context(
                ClientSession(read, write)
            )
            await session.initialize()
            self.sessions.append(session)

            # List available tools for this session
            response = await session.list_tools()
            tools = response.tools
            print(f"\nConnected to {server_name} with tools:", [t.name for t in tools])

            for tool in tools:
                self.tool_to_session[tool.name] = session
                self.available_tools.append({
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                })
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self):
        """Connect to all configured MCP servers."""
        try:
            with open("server_config.json", "r") as file:
                data = json.load(file)

            servers = data.get("mcpServers", {})

            for server_name, server_config in servers.items():
                await self.connect_to_server(server_name, server_config)

            # Initialize chat session after connecting to servers
            await self.initialize_chat()
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise

    async def initialize_chat(self):
        """Initialize the chat session with available tools."""
        gemini_tools = self.convert_mcp_tools_to_genai_format(self.available_tools)
        tools_config = types.Tool(function_declarations=gemini_tools) if gemini_tools else None

        config = types.GenerateContentConfig(tools=[tools_config]) if tools_config else types.GenerateContentConfig()

        self.chat = self.client.chats.create(
            model="gemini-2.5-flash",
            config=config
        )

    async def process_query(self, query):
        # Send message to chat session
        response = self.chat.send_message(query)

        while True:
            # Check if response has function calls
            has_function_call = False
            has_text_response = False

            for part in response.candidates[0].content.parts:
                if hasattr(part, 'function_call') and part.function_call:
                    has_function_call = True
                    function_call = part.function_call
                    tool_name = function_call.name
                    tool_args = dict(function_call.args)

                    print(f"Calling tool {tool_name} with args {tool_args}")

                    # Call tool via correct MCP session
                    session = self.tool_to_session[tool_name]
                    result = await session.call_tool(tool_name, arguments=tool_args)

                    # Send function response to continue conversation
                    function_response = types.FunctionResponse(
                        name=tool_name,
                        response={"result": str(result.content)}
                    )

                    response = self.chat.send_message(
                        types.Part(function_response=function_response)
                    )
                    break  # Process one function call at a time

                elif hasattr(part, 'text') and part.text:
                    has_text_response = True
                    print(part.text)

            # If no function calls, we're done
            if not has_function_call:
                break

    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")

        while True:
            try:
                query = input("\nQuery: ").strip()

                if query.lower() == 'quit':
                    break

                await self.process_query(query)
                print("\n")

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def cleanup(self):
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        # the mcp clients and sessions are not initialized using "with"
        # like in the previous lesson
        # so the cleanup should be manually handled
        await chatbot.connect_to_servers()
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()


if __name__ == "__main__":
    asyncio.run(main())


Overwriting mcp_project/mcp_chatbot.py


## Running the MCP Chatbot

chatbot과 상호작용해 보세요. 다음은 query 예시들입니다:
- Fetch the content of this website: https://modelcontextprotocol.io/docs/concepts/architecture and save the content in the file "mcp_summary.md", create a visual diagram that summarizes the content of "mcp_summary.md" and save it in a text file